In [9]:
%matplotlib inline

import keras 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from collections import Counter

In [150]:
sh = open('Kobzar_1371021036.txt').read().lower()

In [339]:
tokens = [tok for tok in word_tokenize(sh) if not tok in '«[]();:».”?“']
print("Number of tokens:", len(tokens))

Number of tokens: 105414


In [327]:
def ngrams(tokens, n):
    return Counter([tuple(tokens[i:i+n]) for i in range(len(tokens) - (n - 1))] )

In [328]:
g1 = ngrams(tokens, 1)
g2 = ngrams(tokens, 2)
g3 = ngrams(tokens, 3)

In [329]:
g1.most_common(10)

[((',',), 14248),
 (('і',), 2811),
 (('!',), 2662),
 (('не',), 2376),
 (('а',), 1722),
 (('в',), 1675),
 (('на',), 1534),
 (('та',), 1381),
 (('?',), 1034),
 (('з',), 975)]

In [330]:
g2.most_common(10)

[((',', 'і'), 835),
 ((',', 'а'), 713),
 ((',', 'як'), 478),
 ((',', 'що'), 468),
 ((',', 'та'), 456),
 ((',', 'не'), 404),
 (('та', 'й'), 330),
 ((',', '—'), 303),
 (('!', '..'), 253),
 ((',', 'в'), 223)]

In [331]:
def calc_prob(ngram, n_minus_1_ngram):
    probs = {}
    grams = dict(ngram)
    for k1, v1 in grams.items():
        tot = n_minus_1_ngram[tuple(k1[:-1])]
        probs[k1] = v1 / (tot if tot != 0 else len(ngram))
    return probs

In [332]:
p1 = dict(g1)
p2 = calc_prob(g2, g1)
p3 = calc_prob(g3, g2)

In [333]:
gen = ['як', 'його']
N = 20

def generate(N, n, gen, ngrams):
    for i in range(N):
        probs = []
        tokens = []
        for k,prob in ngrams.items():
            if k[:n-1] == tuple(gen[-(n-1):]):
                probs.append(prob)
                tokens.append(k[-1])
        gen.append(np.random.choice(tokens, p=probs, size=(1,))[0])
    return " ".join(gen)

generate(N, 3, gen, p3 )

'як його одній святії прожити літа молодії , сило молодая ! послужи , моя порадо i не видно слава богу ! і'

In [155]:
import string

In [163]:

def rhyme_grams(text):
    rows = [[tok for tok in word_tokenize(row) if not tok in string.punctuation] for row in text.split('\n')]
    bigrams = []
    unigrams = []
    for i in range(len(rows)):
        if rows[i]:
            unigrams.append(rows[i][-1])
        if i >= 2 and rows[i-2] and rows[i]:
            bigrams.append((rows[i-2][-1], rows[i][-1]))
    return Counter(bigrams), Counter(unigrams)

r2, r1 = rhyme_grams(sh)
pr2 = calc_prob(r2, r1)

In [291]:
[(k,v) for k,v in pr2.items() if k[0] == 'хіба']

[(('хіба', 'ні'), 4.7397857616835716e-05)]

In [416]:
gen = ['як',]
N = 50

def generate_poem(N, n, gen, ngrams, size, rows_starts):
    (rows, cols) = size
    poem = []
    for i in range(rows):
        row = rows_starts[i]
        for j in range(cols):
            if j == 0 :
               row.append(sample_from_unigram())
            elif j == 1:
                row.append(sample_from_bigram(row[0]))
            elif j == (cols-1) and  i >= 2:
                probs = []
#                 tokens = []
#                 for k,prob in pr2.items():
#                     if k[0] == poem[i-2][-2]:
#                         probs.append(prob)
#                         tokens.append(k[-1])
                        
                if not probs:
                    probs = []
                    tokens = []
                    for k,prob in ngrams.items():
                        #print(poem[i-2][-2][-len(k[-1][-1:]):], k[-1][-1:])
                        if k[:2-1] == tuple(row[-(2-1):]) and (k[-1][-1:] == poem[i-2][-2][-len(k[-1][-1:]):]):
                            probs.append(prob)
                            tokens.append(k[-1])
                    #print(poem)
                    print(poem[i-2][-2][-len(k[-1][-1:]):])
                    if not probs:
                        probs = []
                        tokens = []
                        for k,prob in g1.items():
                            if k[-1][-1:] == poem[i-2][-2][-len(k[-1][-1:]):]:
                                probs.append(prob)
                                tokens.append(k[-1])
                        row.append(np.random.choice(tokens, p=normalize(probs), size=(1,))[0])
                    else:
                        row.append(np.random.choice(tokens, p=normalize(probs), size=(1,))[0])
                else:
                    se = np.random.choice(tokens, p=normalize(probs), size=(1,))[0]
                    print(poem[i-2][-2], se)
                    row.append(se)
            else:
                probs = []
                tokens = []
                for k,prob in ngrams.items():
                    if k[:n-1] == tuple(row[-(n-1):]):
                        probs.append(prob)
                        tokens.append(k[-1])
                        
                if not probs:
                    probs = []
                    tokens = []
                    for k,prob in ngrams.items():
                        if k[:1] == tuple(row[-(2-1):]):
                            probs.append(prob)
                            tokens.append(k[-1])

                    row.append(np.random.choice(tokens, p=normalize(probs), size=(1,))[0])
                else:
                    row.append(np.random.choice(tokens, p=normalize(probs), size=(1,))[0])
        row.append('\n')
        poem.append(row)
    return poem

print(" ".join([" ".join(row) for row in generate_poem(N, 3, gen, p3, (4,5), [['куди', 'піти'], ['чого','немає'], ['мені','дає'], ['він','хоче']])]))

о
,
куди піти я гне посумує сидячи соловейко 
 чого немає пісок се сподіється чи , 
 мені дає мій на отой проклятий що 
 він хоче україну , де його , 



In [345]:
def normalize(xs):
    s = sum(xs)
    return [x/s for x in xs]

In [209]:
def sample_from_unigram():
    probs = []
    tokens = []
    for k,v in list(g1.items())[100:]:
        probs.append(v)
        tokens.append(k[0])
    return np.random.choice(tokens, p=normalize(probs), size=(1,))[0]
        
sample_from_unigram()

'світі'

In [208]:
def sample_from_bigram(prev):
    probs = []
    tokens = []
    for k,v in list(g2.items())[100:]:
        if k[0] == prev:
            probs.append(v)
            tokens.append(k[-1])
    return np.random.choice(tokens, p=normalize(probs), size=(1,))[0]

sample_from_bigram('я')

','